<a href="https://colab.research.google.com/github/Mirra-mira/Backttracking/blob/main/Nhom05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install simpleai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.4/94.4 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for simpleai: filename=simpleai-0.8.3-py3-none-any.whl size=100982 sha256=d624cac82c8718f0776c2647361afc7d5090ac70f8fc9d2673e854142b236ed5
  Stored in directory: /root/.cache/pip/wheels/ec/02/a7/f0077617a5f73eb1c52e45f12a9da3f0bafff3902bcd91766f
Successfully built simpleai


In [ ]:
from simpleai.search import CspProblem, backtrack
import random
import math

def show_constraints():
    print("\n*** Ràng buộc tối thiểu ***")
    print("- Số buổi học cố định là 12 (6 ngày x 2 buổi/ngày).")
    print("- Số phòng học tối thiểu: 13 lớp cần ít nhất 2 phòng.")
    print("- Số giáo viên tối thiểu: 13 lớp cần ít nhất 2 giáo viên.")
    print("- Số lớp học không nên vượt quá (số phòng học x 12) để đảm bảo khả thi.")
    print("***************************************\n")

def schedule_classes():
    # Người dùng nhập dữ liệu
    num_classes = int(input("Nhập số lượng lớp học: "))
    num_teachers = int(input("Nhập số lượng giáo viên: "))
    subjects_input = input("Nhập danh sách môn học (cách nhau bởi dấu phẩy): ")
    num_rooms = int(input("Nhập số lượng phòng học: "))

    # Kiểm tra tính khả thi sơ bộ
    min_rooms = math.ceil(num_classes / 12)
    min_teachers = math.ceil(num_classes / 12)
    max_classes = num_rooms * 12

    if num_rooms < min_rooms:
        print(f"Cảnh báo: Cần ít nhất {min_rooms} phòng học cho {num_classes} lớp!")
    if num_teachers < min_teachers:
        print(f"Cảnh báo: Cần ít nhất {min_teachers} giáo viên cho {num_classes} lớp!")
    if num_classes > max_classes:
        print(f"Cảnh báo: Với {num_rooms} phòng, tối đa chỉ xếp được {max_classes} lớp!")

    # Xử lý danh sách môn học
    subjects = [subject.strip() for subject in subjects_input.split(',')]
    if not subjects:
        print("Danh sách môn học không hợp lệ!")
        return

    # Kiểm tra số lượng phòng học
    if num_rooms <= 0:
        print("Số lượng phòng học phải lớn hơn 0!")
        return

    # Khởi tạo dữ liệu
    rooms = [f'Phong{i}' for i in range(1, num_rooms + 1)]  # Danh sách phòng học
    # Định nghĩa 12 buổi học: Sáng/Chiều từ thứ 2 đến thứ 7
    times = [
        'Sáng thứ 2', 'Chiều thứ 2',
        'Sáng thứ 3', 'Chiều thứ 3',
        'Sáng thứ 4', 'Chiều thứ 4',
        'Sáng thứ 5', 'Chiều thứ 5',
        'Sáng thứ 6', 'Chiều thứ 6',
        'Sáng thứ 7', 'Chiều thứ 7'
    ]
    classes = [f'Lop{i}' for i in range(1, num_classes + 1)]  # Danh sách lớp học
    teachers = [f'GV{i}' for i in range(1, num_teachers + 1)]  # Danh sách giáo viên

    # Gán ngẫu nhiên giảng viên và môn học cho từng lớp
    random.seed(42)  # Đảm bảo kết quả tái lập
    class_info = {}
    for cls in classes:
        class_info[cls] = {
            'teacher': random.choice(teachers),
            'subject': random.choice(subjects)
        }

    # Biến: Mỗi lớp học là một biến
    variables = classes

    # Miền giá trị: Mỗi lớp học có thể được gán một cặp (khung giờ, phòng học)
    domains = {}
    for cls in variables:
        domains[cls] = [(time, room) for time in times for room in rooms]

    # Ràng buộc
    constraints = []

    # Ràng buộc 1: Không có hai lớp học nào cùng khung giờ và cùng phòng học
    def constraint_different_time_room(variables, values):
        time1, room1 = values[0]
        time2, room2 = values[1]
        return (time1 != time2) or (room1 != room2)

    # Thêm ràng buộc cho từng cặp lớp học
    for i in range(len(classes)):
        for j in range(i + 1, len(classes)):
            constraints.append(((classes[i], classes[j]), constraint_different_time_room))

    # Ràng buộc 2: Không có hai lớp học nào do cùng giảng viên dạy được gán cùng khung giờ
    def constraint_different_teacher_time(variables, values):
        cls1, cls2 = variables
        time1, _ = values[0]
        time2, _ = values[1]
        teacher1 = class_info[cls1]['teacher']
        teacher2 = class_info[cls2]['teacher']
        if teacher1 == teacher2:
            return time1 != time2
        return True

    # Thêm ràng buộc giảng viên
    for i in range(len(classes)):
        for j in range(i + 1, len(classes)):
            if class_info[classes[i]]['teacher'] == class_info[classes[j]]['teacher']: #fixed this line
                constraints.append(((classes[i], classes[j]), constraint_different_teacher_time))

    # Tạo bài toán CSP
    problem = CspProblem(variables, domains, constraints)

    # Giải bài toán bằng Backtracking
    solution = backtrack(problem)

    # In kết quả
    if solution:
        print("\nLịch học được sắp xếp:")
        for cls, (time, room) in solution.items():
            teacher = class_info[cls]['teacher']
            subject = class_info[cls]['subject']
            print(f"{cls} ({subject}, {teacher}): {time} tại {room}")
    else:
        print("\nKhông tìm được lịch học phù hợp.")

def main():
    while True:
        print("\n=== Chương trình xếp lịch học ===")
        print("1. Xem ràng buộc")
        print("2. Bắt đầu xếp lớp")
        print("0. Thoát")
        choice = input("Nhập lựa chọn (0-2): ")

        if choice == '1':
            show_constraints()
        elif choice == '2':
            schedule_classes()
        elif choice == '0':
            print("Tạm biệt!")
            break
        else:
            print("Lựa chọn không hợp lệ! Vui lòng chọn lại.")

if __name__ == "__main__":
    main()
    # Tạo bài toán CSP
    problem = CspProblem(variables, domains, constraints)

    # Giải bài toán bằng Backtracking
    solution = backtrack(problem)

    # In kết quả
    if solution:
        print("\nLịch học được sắp xếp:")
        for cls, (time, room) in solution.items():
            teacher = class_info[cls]['teacher']
            subject = class_info[cls]['subject']
            print(f"{cls} ({subject}, {teacher}): {time} tại {room}")
    else:
        print("\nKhông tìm được lịch học phù hợp.")